In [1]:
import open3d as o3d
import pandas as pd
import numpy as np
import random
import io
import matplotlib.pyplot as plt
import sys

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def read_and_preprocess(filepath, labelspath, factor=0.3):
    pc = pd.read_csv(filepath, sep=' ',header=None,  dtype=np.float32).to_numpy()
    print(pc.shape)
    pc_labels_unfil = pd.read_csv(labelspath).to_numpy()
    non_zero_indices = np.where(pc_labels_unfil != 0)
    pc_xyz_u = pc
    pc_xyz_u = pc_xyz_u[non_zero_indices[0], :]
    pc_labels = pc_labels_unfil[non_zero_indices]
    pc_xyz = pc_xyz_u[:, 0:3]
    pc_rgb = pc_xyz_u[:, 4:]
    randints = np.random.randint(0, high = pc_xyz.shape[0], size = int(factor*pc_xyz.shape[0]))
    pc_xyz = pc_xyz[randints, :]
    pc_rgb = pc_rgb[randints, :]
    #randints = 0
    pc_labels = pc_labels[randints]
    #print(pc_labels)
    return pc_xyz, pc_labels, pc_rgb, randints, non_zero_indices

def read_probs(filepath, randints, nonzero_indices):
    probs, labels = np.load(filepath)
    probs = probs[nonzero_indices[0]]
    labels = labels[nonzero_indices[0]]
    probs_downsampled = probs[randints]
    labels_downsampled = labels[randints]
    return probs_downsampled, labels_downsampled
    return probs, labels

In [3]:
def labels_to_colors(pc_labels):
    cmap = {0: [0, 0, 0],
            1: [0, 0, 1],
            2: [0, 1, 0],
            3: [1, 0, 0],
            4: [0, 1, 1],
            5: [1, 0, 1],
            6: [1, 1, 0],
            7: [0, 0.5, 0.5],
            8: [0.5, 0, 0.5]}
    print(pc_labels.shape)
    pc_clrs = np.asarray([cmap[x] for x in pc_labels])
    return pc_clrs

def convert_prob_colors(pc_probs, thold):
    pc_clrs = []
    for prob in pc_probs:
        if prob >= thold:
            pc_clrs.append([0, 1, 0])
        else:
            pc_clrs.append([1, 0, 0])
    return np.asarray(pc_clrs)

def convert_ent_colors(pc_ent, thold):
    pc_clrs = []
    for ent in pc_ent:
        if ent <= thold:
            pc_clrs.append([0, 1, 0])
        else:
            pc_clrs.append([1, 0, 0])
    return np.asarray(pc_clrs)

# Semantic3D arguments
## MSP OOD Plots
1. Ensembles/Flipout passes: 10
2. Ensembles: 0.755004
3. Flipout: 0.716796

In [4]:
path = './bildstein_station1_xyz_intensity_rgb.txt'
label_path = './bildstein_station1_xyz_intensity_rgb.labels'
max_prob_path = './sem3d/10/bildstein_station1_xyz_intensity_rgb_probs.npy'
pc_xyz, pc_labels, pc_rgb, downsampled_indices, nonzero_indices = read_and_preprocess(path, label_path, factor=0.3)
pc_probs, pc_labels_detect = read_probs(max_prob_path, downsampled_indices, nonzero_indices)
pc_clrs = convert_prob_colors(pc_probs, 0.755004)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_xyz)
pcd.colors = o3d.utility.Vector3dVector(pc_clrs)
o3d.visualization.draw_geometries([pcd])

(29697591, 7)


In [6]:
max_prob_path = './max_probs_flipout/sem3d/10/bildstein_station1_xyz_intensity_rgb_probs.npy'
pc_probs, pc_labels_detect = read_probs(max_prob_path, downsampled_indices, nonzero_indices)
pc_clrs = convert_prob_colors(pc_probs, 0.716796)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_xyz)
pcd.colors = o3d.utility.Vector3dVector(pc_clrs)
o3d.visualization.draw_geometries([pcd])

## Entropy OOD Plots
1. Ensembles: 0.386315
2. Flipout: 0.438823

In [7]:
max_prob_path = './entropy_val/entropies/sem3d/10/bildstein_station1_xyz_intensity_rgb_probs.npy'
#max_prob_path = './entropies_flipout/sem3d/20/bildstein_station1_xyz_intensity_rgb_probs.npy'
pc_probs, pc_labels_detect = read_probs(max_prob_path, downsampled_indices, nonzero_indices)
pc_clrs = convert_ent_colors(pc_probs, 0.386315)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_xyz)
pcd.colors = o3d.utility.Vector3dVector(pc_clrs)
o3d.visualization.draw_geometries([pcd])

In [9]:
max_prob_path = './entropies_flipout/sem3d/10/bildstein_station1_xyz_intensity_rgb_probs.npy'
pc_probs, pc_labels_detect = read_probs(max_prob_path, downsampled_indices, nonzero_indices)
pc_clrs = convert_ent_colors(pc_probs, 0.438823)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pc_xyz)
pcd.colors = o3d.utility.Vector3dVector(pc_clrs)
o3d.visualization.draw_geometries([pcd])